In [1]:
import numpy as np
import pandas as pd
import time 
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder

E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


**读入数据:**

In [2]:
#a1 = OneHotEncoder(sparse = False).fit_transform( trains[['用户性别']] )
#a2 = OneHotEncoder(sparse = False).fit_transform( trains[['用户职业']])
#a3 = OneHotEncoder(sparse = False).fit_transform( trains[['用户教育程度']])
#a4 = OneHotEncoder(sparse = False).fit_transform( trains[['用户婚姻状态']])
#a5 = OneHotEncoder(sparse = False).fit_transform( trains[['用户户口类型']])
#final_output = np.hstack((a1,a2,a3,a4,a5))
#final_output

#pandas 自带的get_dummies函数实现one-hot,并构造组合特征
# trains = pd.read_csv("../feature/训练表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
# #trains.index = trains['用户标识'].tolist()
# train_label=trains[['用户标识','标签']]
# trains=pd.get_dummies(trains,columns=trains[['用户性别','用户职业','用户教育程度','用户婚姻状态','用户户口类型']]).drop(['标签'],axis=1)

# tests = pd.read_csv("../feature/测试表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
# #tests.index = tests['用户标识'].tolist()
# test_label=tests[['用户标识','标签']]
# tests=pd.get_dummies(tests,columns=tests[['用户性别','用户职业','用户教育程度','用户婚姻状态','用户户口类型']]).drop(['标签'],axis=1)

# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(interaction_only=True)
# trains=poly.fit_transform(trains)
# df = pd.DataFrame(trains.reshape(55596, 326))
# df.interpolate(axis=1).values.reshape(trains.shape)
# df['用户标识']=df[1].astype(int)
# trains=pd.merge(train_label,df,how='inner', on = "用户标识")

# tests=poly.fit_transform(tests)
# df = pd.DataFrame(tests.reshape(13899, 326))
# df.interpolate(axis=1).values.reshape(tests.shape)
# df['用户标识']=df[1].astype(int)
# tests=pd.merge(test_label,df,how='inner', on = "用户标识")

In [159]:
dataset.shape

(69495, 326)

# 用户信息表：  
69495 rows × 303 columns

In [4]:
train = pd.read_csv("../feature/训练表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("../feature/测试表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
test=test.fillna(-1)

dataset= pd.concat([train,test],axis=0).reset_index()
label=dataset[['用户标识','标签']]
dataset=dataset.drop(['index','用户标识'],axis=1)

dataset=pd.get_dummies(dataset,columns=dataset[['用户性别','用户职业','用户教育程度','用户婚姻状态','用户户口类型']]).drop(['标签'],axis=1)
dataset #69495 rows × 24 columns
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True)
dataset=poly.fit_transform(dataset)
df = pd.DataFrame(dataset.reshape(69495,301))
df.interpolate(axis=1).values.reshape(dataset.shape)

#dataset
#trains=pd.merge(label,df,left_index=True,right_index=True,how='outer')
trains=pd.concat([label,df],axis=1)
trains#69495 rows × 303 columns

,用户标识,标签,0,1,2,3,4,5,6,7,...,291,292,293,294,295,296,297,298,299,300
0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 用户浏览行为训练表：  
69495 rows × 343 columns

In [5]:
train = pd.read_csv("../feature/用户浏览行为测试表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("../feature/用户浏览行为训练表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户浏览行为缺失统计']=(dataset==-1).sum(axis=1)
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
trains=pd.merge(trains,temp,how='left', on = "用户标识")
trains#69495 rows × 343columns

,用户标识,标签,0,1,2,3,4,5,6,7,...,放款后浏览子行为编号_3,放款后浏览子行为编号_4,放款后浏览子行为编号_5,放款后浏览子行为编号_6,放款后浏览子行为编号_7,放款后浏览子行为编号_8,放款后浏览子行为编号_9,放款后浏览子行为编号_10,放款后浏览子行为编号_11,用户浏览行为缺失统计
0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38
1,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,90.0,150.0,105.0,135.0,105.0,105.0,0.0,75.0,0.0,8
2,3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,48.0,30.0,24.0,42.0,0.0,0.0,24.0,0.0,8
3,4,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4.0,64.0,36.0,32.0,36.0,4.0,0.0,24.0,0.0,8
4,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38
5,6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,8
6,7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,140.0,98.0,98.0,112.0,14.0,0.0,84.0,0.0,8
7,8,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,22.0,154.0,66.0,99.0,132.0,11.0,0.0,132.0,0.0,8
8,9,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38
9,10,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,20.0,40.0,20.0,20.0,30.0,40.0,0.0,20.0,0.0,8


# 用户银行流水记录表：  
69495 rows × 369  columns

In [6]:
train = pd.read_csv("../feature/用户银行流水记录训练表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("../feature/用户银行流水记录测试表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
#temp
trains=pd.merge(trains,temp,how='left', on = "用户标识")
trains#69495 rows × 369  columns

,用户标识,标签,0,1,2,3,4,5,6,7,...,放款后用户支出总计,放款后用户工资收入笔数,放款后用户工资收入总计,放款后用户收入支出笔数差值,放款后用户收入支出总计差值,放款后用户非工资收入笔数,放款后用户非工资收入总计,放款后工资收入笔数乘以差值,放款后工资收入总计乘以差值,用户银行流水记录缺失统计
0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
1,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
2,3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
3,4,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
4,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
5,6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
6,7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
7,8,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
8,9,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
9,10,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,981.631241,NaN,NaN,-7.0,-83.078408,NaN,NaN,NaN,NaN,12


# 用户账单表初级特征:  
69495 rows × 448  columns

In [7]:
train = pd.read_csv("../feature/用户账单表初级特征训练表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("../feature/用户账单表初级特征测试表_20170119_A.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
print(temp.shape)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#特征轮：构造放款后与放款前差值特征
feature_name=temp.columns 
for name in feature_name:
    if name.find('款后')> 0:
        x1=name.replace('款后','款前')
        if x1 in feature_name:
            temp[name+"与"+x1+"差值"]=temp[name]-temp[x1]        
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
#temp

trains=pd.merge(trains,temp,how='left', on = "用户标识")
trains#69495 rows × 448  columns

(69495, 57)


,用户标识,标签,0,1,2,3,4,5,6,7,...,用户放款后消费笔数(去重)与用户放款前消费笔数(去重)差值,用户放款后循环利息(去重)与用户放款前循环利息(去重)差值,放款后账单大于还款计数(去重)与放款前账单大于还款计数(去重)差值,放款后账单等于还款计数(去重)与放款前账单等于还款计数(去重)差值,放款后账单小于还款计数(去重)与放款前账单小于还款计数(去重)差值,放款后账单汇总还款差额(去重)与放款前账单汇总还款差额(去重)差值,放款后账单最大值还款差额(去重)与放款前账单最大值还款差额(去重)差值,用户放款后消费笔数与用户放款前消费笔数差值,用户放款后循环利息与用户放款前循环利息差值,用户放款后信用卡额度最大值与用户放款前信用卡额度最大值差值
0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,11.0,0.000000,1.0,NaN,3.0,6.638937,6.638937,11.0,0.000000,1.609437
2,3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,1.0,NaN,NaN,1.181840,1.181840,0.0,0.000000,-0.154151
4,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-35.0,0.000000,NaN,NaN,0.0,71.735393,15.957393,-200.0,0.000000,-0.287682
7,8,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-41.0,0.000000,-9.0,NaN,-15.0,-6.357178,2.182406,-90.0,0.000000,0.693147


# 用户账单表特征:  
69495 rows × 1146 columns


In [8]:
train = pd.read_csv("../feature/用户账单表特征训练表20170119_A.csv",encoding="gb2312") # 注意自己数据路径
test = pd.read_csv("../feature/用户账单表特征测试表20170119_A.csv",encoding="gb2312") # 注意自己数据路径
dataset= pd.concat([train,test],axis=0)
temp=dataset[:]
print(temp.shape)#(69495, 466)
#统计行和列的缺失值数
dataset=dataset.fillna(-1)
temp['用户银行流水记录缺失统计']=(dataset==-1).sum(axis=1)
#特征轮：构造放款后与放款前差值特征
feature_name=temp.columns 
for name in feature_name:
    if name.find('款后')> 0:
        x1=name.replace('款后','款前')
        if x1 in feature_name:
            temp[name+"与"+x1+"差值"]=temp[name]-temp[x1]        
#用均值填充缺失值
#temp.fillna(temp.mean(),inplace=True)
#temp#69495 rows × 699 columns

trains=pd.merge(trains,temp,how='left', on = "用户标识")
trains=trains.rename(index=str, columns={"用户标识": "userid"})
trains#69495 rows × 1146 columns

(69495, 466)


,用户标识,标签,0,1,2,3,4,5,6,7,...,去重后放款后消费笔数var与去重后放款前消费笔数var差值,去重后放款后本期账单金额var与去重后放款前本期账单金额var差值,去重后放款后调整金额var与去重后放款前调整金额var差值,去重后放款后循环利息var与去重后放款前循环利息var差值,去重后放款后可用余额var与去重后放款前可用余额var差值,去重后放款后预借现金额度var与去重后放款前预借现金额度var差值,去重后放款后上期还款金额var与放款后上期账单金额var差值与去重后放款前上期还款金额var与放款前上期账单金额var差值差值,去重后放款后信用卡额度var与放款后本期账单余额var差值与去重后放款前信用卡额度var与放款前本期账单余额var差值差值,去重后放款后可用余额var与放款后预借现金额度var差值与去重后放款前可用余额var与放款前预借现金额度var差值差值,去重后放款后本期账单最低还款额var与放款后循环利息var之和与去重后放款前本期账单最低还款额var与放款前循环利息var之和差值
0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-1.119658,12.825337,0.000000,0.000000,0.000000,70.266540,-2.237091,-68.591951,-70.266540,-113.517191
2,3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.200000,-6.149673,0.000000,0.000000,-31.789011,-3.275160,-12.924828,26.717870,-28.513851,-0.454825
4,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-8.628151,7.711654,0.000000,0.000000,10.795729,1.814566,262.768115,314.696835,8.981163,-75.921730
7,8,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-1.613749,-38.092504,0.000000,0.000000,-15.281851,11.574647,-17.033798,-19.116719,-26.856498,-130.122182


In [15]:
dataset=trains[:]
dataset.drop('放款时间_x',axis=1, inplace=True)
dataset.drop('放款时间_y',axis=1, inplace=True)
dataset.shape

E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


(69495, 1142)

In [16]:
test=dataset[(dataset['标签']<0)]#测试集:13899行
train=dataset[(dataset['标签']>=0)]#训练集:55596
train = train.drop(['标签'],axis=1)
test = test.drop(['标签'],axis=1)
train

,userid,0,1,2,3,4,5,6,7,8,...,去重后放款后消费笔数var与去重后放款前消费笔数var差值,去重后放款后本期账单金额var与去重后放款前本期账单金额var差值,去重后放款后调整金额var与去重后放款前调整金额var差值,去重后放款后循环利息var与去重后放款前循环利息var差值,去重后放款后可用余额var与去重后放款前可用余额var差值,去重后放款后预借现金额度var与去重后放款前预借现金额度var差值,去重后放款后上期还款金额var与放款后上期账单金额var差值与去重后放款前上期还款金额var与放款前上期账单金额var差值差值,去重后放款后信用卡额度var与放款后本期账单余额var差值与去重后放款前信用卡额度var与放款前本期账单余额var差值差值,去重后放款后可用余额var与放款后预借现金额度var差值与去重后放款前可用余额var与放款前预借现金额度var差值差值,去重后放款后本期账单最低还款额var与放款后循环利息var之和与去重后放款前本期账单最低还款额var与放款前循环利息var之和差值
0,1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.119658,12.825337,0.000000,0.000000,0.000000,70.266540,-2.237091,-68.591951,-70.266540,-113.517191
2,3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.200000,-6.149673,0.000000,0.000000,-31.789011,-3.275160,-12.924828,26.717870,-28.513851,-0.454825
4,5,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-8.628151,7.711654,0.000000,0.000000,10.795729,1.814566,262.768115,314.696835,8.981163,-75.921730
7,8,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.613749,-38.092504,0.000000,0.000000,-15.281851,11.574647,-17.033798,-19.116719,-26.856498,-130.122182


In [182]:
#trains.to_csv("../feature/特征汇总_20170119_A.csv",index=None,encoding="gb2312") #mean填充 69495 rows × 1171 columns 没有删去用户标识
#trains.to_csv("../feature/特征汇总_20170119_B.csv",index=None,encoding="gb2312") #0填充  69495 rows × 1171 columns 没有删去用户标识

#trains.to_csv("../feature/特征汇总_20170119_D.csv",index=None,encoding="gb2312") #0填充 69495 rows × 1146 columns 当前最高分0.43267
#trains.to_csv("../feature/特征汇总_20170119_E.csv",index=None,encoding="gb2312") #mean填充 69495 rows × 1146 columns

In [183]:
if "放款时间" in names:
    print("ok")
else:
    print("sorry")

sorry


# 特征得分高的特征进行多项式组合log(x*y)

In [8]:
dataset= pd.read_csv("../feature/特征汇总_20170119_D.csv",encoding="gb2312") # 注意自己数据路径
feature_score=pd.read_csv("../feature_score/feature_score_20170119_D.csv",encoding="gb2312")
feature_list=feature_score[(feature_score.score>800)].feature.tolist()
dataset_combine=dataset[feature_list]

dataset1=dataset[['用户标识','标签']]
dataset.drop('放款时间_x',axis=1, inplace=True)
dataset.drop('放款时间_y',axis=1, inplace=True)
dataset.drop('放款时间_x.1',axis=1, inplace=True)
dataset.drop('放款时间_y.1',axis=1, inplace=True)
from sklearn.preprocessing import MinMaxScaler
#dataset=trains[:]
#检测重复特征名称,并删除
names=dataset.columns.tolist()
print(len(names))
print(len(set(names)))
print("dataset大小：")
print(dataset.shape)
myset = set(names)
for item in myset:
    if item=='标签':
        continue
    if item=='用户标识':
        continue
    #dataset[item] = MinMaxScaler().fit_transform(dataset[item])
    if names.count(item)>1:
        print(item)
        dataset=dataset.drop(item,axis=1)
        #dataset.rename(columns={item:11}, inplace = True)
    dataset1['原始特征'+item]=dataset[item]
print("新dataset1大小：")
print(dataset1.shape)
dataset=dataset1
dataset1=0

1142
1142
dataset大小：
(69495, 1142)


E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


新dataset1大小：
(69495, 1142)


In [6]:
#a.rename(columns={'A':'a', 'B':'b', 'C':'c'}, inplace = True)

In [9]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True)
dataset_combine=poly.fit_transform(dataset_combine)
df = pd.DataFrame(dataset_combine.reshape(69495,991))
df.interpolate(axis=1).values.reshape(dataset_combine.shape)

#dataset
dataset=pd.concat([dataset,df],axis=1)
dataset#69495 rows × 2133 columns

,用户标识,标签,原始特征去重后放款后本期账单金额std与去重后放款前本期账单金额std差值,原始特征158,原始特征放款后上期账单金额std,原始特征放款后账单大于还款计数(去重),原始特征147,原始特征45,原始特征204,原始特征放款后信用卡额度sum与放款后本期账单余额sum差值,...,981,982,983,984,985,986,987,988,989,990
0,1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,0.0,-0.345102,0.0,9.736646,3.0,0.0,0.0,0.0,6.275953,...,-67.020321,-67.009714,-730.088252,11.997253,374.336736,4078.496050,-67.020321,4077.850550,-67.009714,-730.088252
2,3,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.248848,0.248848,0.000014,-0.000125,336.659030,0.019002,-0.168749,0.019002,-0.168749,-0.000010
3,4,1.0,0.542952,0.0,13.419102,4.0,0.0,0.0,0.0,-66.198175,...,-32.742107,-31.982164,-1.999009,2.944795,355.598070,22.226258,-32.742107,21.710388,-31.982164,-1.999009
4,5,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.585486,0.585486,0.000086,-0.000345,425.846395,0.062857,-0.251017,0.062857,-0.251017,-0.000037
5,6,1.0,0.000000,0.0,11.235244,0.0,0.0,0.0,0.0,0.000000,...,27.986054,26.687733,11.834550,3.596358,385.610972,170.997375,51.963760,163.064511,49.553071,21.974077
6,7,0.0,4.372128,0.0,11.701114,1.0,0.0,0.0,0.0,36.446441,...,258.062302,38.569103,1604.738101,237.730736,32.633597,1357.780516,201.145696,202.929198,30.062543,1250.807110
7,8,0.0,0.000000,0.0,0.865703,0.0,0.0,0.0,0.0,0.006492,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,9,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,21.952360,19.155954,33.267824,1.433238,349.521956,607.008920,26.151038,529.684971,22.819783,39.630734
9,10,0.0,0.000000,0.0,0.000000,13.0,0.0,0.0,0.0,189.637897,...,71.431588,60.477364,688.673268,15.671407,291.038223,3314.136548,75.416290,2805.904932,63.851002,727.089855


In [10]:
tests=dataset[(dataset['标签']<0)]#测试集:13899行
trains=dataset[(dataset['标签']>=0)]#训练集:55596
trains.shape

(55596, 2133)

In [ ]:
#trains=trains.drop(['放款时间'],axis=1) #55596行  
#tests=tests.drop(['放款时间'],axis=1)#测试集:13899行
online_test=tests
train_xy,offline_test = train_test_split(trains, test_size = 0.25,random_state=36)#41697/13899
train,val = train_test_split(train_xy, test_size = 0.333,random_state=24)#27811/13886
#train,val = train_test_split(trains, test_size = 0.25,random_state=20)#41697/13899
#train_xy,offline_test = train_test_split(trains, test_size = 0.25,random_state=10)#41697/13899

# KS评测:

In [12]:
#t1=trains.columns
#t2=tests.columns
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    #label = y_true.get_label()
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()

# CV:

http://scikit-learn.org/stable/modules/cross_validation.html

In [29]:
from sklearn.cross_validation import KFold
kf = KFold(10, n_folds=5,shuffle=True)
print(k_fold) 
for train_index, test_index in kf:
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

KFold(n_splits=4, random_state=None, shuffle=True)
TRAIN: [0 2 3 4 6 7 8 9] TEST: [1 5]
TRAIN: [0 1 3 4 5 6 7 8] TEST: [2 9]
TRAIN: [0 1 2 3 5 6 8 9] TEST: [4 7]
TRAIN: [0 1 2 4 5 6 7 9] TEST: [3 8]
TRAIN: [1 2 3 4 5 7 8 9] TEST: [0 6]


In [31]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=3,shuffle=True)
for train_indices, test_indices in k_fold.split(X):
    print('Train: %s | test: %s' % (train_indices, test_indices))

Train: [    0     1     4 ..., 31268 31269 31271] | test: [    2     3     8 ..., 31264 31267 31270]
Train: [    2     3     4 ..., 31267 31269 31270] | test: [    0     1     5 ..., 31266 31268 31271]
Train: [    0     1     2 ..., 31268 31270 31271] | test: [    4     7     9 ..., 31254 31262 31269]


In [49]:
from sklearn import datasets
iris = datasets.load_iris()
from sklearn import svm
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, iris.data, iris.target, cv=4)
scores

array([ 1.        ,  0.97435897,  0.97222222,  0.97222222])

In [15]:
cvresult =xgb.cv(plst, xgb_train,num_boost_round=3500,early_stopping_rounds=500, nfold=5, maximize=False, verbose_eval=True)

[0]	train-auc:0.767094+0.00228847	test-auc:0.751064+0.00536723
[1]	train-auc:0.775101+0.00168136	test-auc:0.76027+0.00415486
[2]	train-auc:0.778371+0.00192122	test-auc:0.763009+0.00410371
[3]	train-auc:0.780969+0.00225017	test-auc:0.764665+0.00500301
[4]	train-auc:0.782489+0.00160775	test-auc:0.765751+0.00643715
[5]	train-auc:0.782789+0.00179156	test-auc:0.766438+0.00664493
[6]	train-auc:0.783141+0.00156306	test-auc:0.766709+0.00679769
[7]	train-auc:0.7835+0.001416	test-auc:0.767159+0.00676144
[8]	train-auc:0.783993+0.0010929	test-auc:0.767543+0.00697299
[9]	train-auc:0.784503+0.000864349	test-auc:0.767981+0.00672795
[10]	train-auc:0.784761+0.000992461	test-auc:0.768425+0.00629241
[11]	train-auc:0.784832+0.000779227	test-auc:0.768917+0.00651946
[12]	train-auc:0.785082+0.00107424	test-auc:0.769022+0.00660513
[13]	train-auc:0.785326+0.00108801	test-auc:0.769229+0.0065952
[14]	train-auc:0.785393+0.00104472	test-auc:0.769266+0.00649872
[15]	train-auc:0.785557+0.000925843	test-auc:0.769116+

# **模型:**

In [15]:
start_time = time.time()
offline=0
online=0
params={'booster':'gbtree',
    'objective': 'rank:pairwise',
    'eval_metric':'auc',
    'gamma':0.1,
    'min_child_weight':1.1,
    'max_depth':7,
    'lambda':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'colsample_bylevel':0.7,
    'eta': 0.01,
    'tree_method':'exact',
    'seed':1000,
    'nthread':12
    }

params1={
'booster':'gbtree',
'objective': 'binary:logistic',
'scale_pos_weight': 1/7.5,
#7183正样本
#55596条总样本
#差不多1:7.7这样子
'gamma':0.2,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':8, # 构建树的深度，越大越容易过拟合
'lambda':3,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
#'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.03, # 如同学习率
'seed':1000,
'nthread':12,# cpu 线程数
'eval_metric': 'auc'
}

plst = list(params.items())
num_rounds = 5000 # 迭代次数

y = train.标签
X = train.drop(['标签','用户标识'],axis=1)
#X=train[feature_list]

val_y = val.标签
val_X = val.drop(['标签','用户标识'],axis=1)
#val_X = val[feature_list]

offline_test_X=offline_test.drop(['标签','用户标识'],axis=1)
online_test_X=online_test.drop(['标签','用户标识'],axis=1)
#offline_test_X=offline_test[feature_list]
#online_test_X=online_test[feature_list]

xgb_train = xgb.DMatrix(X, label=y)
xgb_val = xgb.DMatrix(val_X,label=val_y)

xgb_offline_test = xgb.DMatrix(offline_test_X)
xgb_online_test = xgb.DMatrix(online_test_X)

# return 训练和验证的错误率
watchlist = [(xgb_train, 'train'),(xgb_val, 'val')]

print ("跑到这里了xgb.train")
# training model 
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train,num_boost_round=7000,evals=watchlist,early_stopping_rounds=500)
print ("跑到这里了save_model")
model.save_model('../model/20170201_B.model') # 用于存储训练出的模型
print ("best best_ntree_limit",model.best_ntree_limit)   #did not save the best,why?
print ("best best_iteration",model.best_iteration) #get it?

print ("跑到这里了model.predict")
preds_offline = model.predict(xgb_offline_test,ntree_limit=model.best_iteration)#
preds_online = model.predict(xgb_online_test,ntree_limit=model.best_iteration)#
offline=offline_test[['用户标识','标签']]
online=online_test[['用户标识']]

offline['预测']=preds_offline
online['预测']=preds_online

print("线下得分;")
print(ks(offline.预测,offline.标签))

print ("跑到这里了,输出结果")
from sklearn.preprocessing import MinMaxScaler
online.预测 = MinMaxScaler().fit_transform(online.预测)
output=online[['用户标识','预测']].rename(index=str, columns={"用户标识": "userid", "预测": "probability"})
output.to_csv("../output/20170201_B.csv",index=None,encoding='utf-8')

#save feature score and feature information:  feature,score,min,max,n_null,n_gt1w
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('../feature_score/feature_score_20170201_B.csv','w') as f:
        f.writelines("feature,score\n")
        f.writelines(fs)
        
cost_time = time.time()-start_time
print ("",'\n',"cost time:",cost_time,"(s)")
print ("跑到这里了,结束！")
# CV训练
#xgb.cv(xgb_config, dtrain, num_boost_round=10, nfold=5, maximize=False, verbose_eval=True)

跑到这里了xgb.train
[0]	train-auc:0.788078	val-auc:0.743271
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 500 rounds.
[1]	train-auc:0.805151	val-auc:0.75708
[2]	train-auc:0.811012	val-auc:0.76598
[3]	train-auc:0.815672	val-auc:0.770599
[4]	train-auc:0.818262	val-auc:0.771523
[5]	train-auc:0.820617	val-auc:0.772233
[6]	train-auc:0.821665	val-auc:0.771915
[7]	train-auc:0.821911	val-auc:0.771823
[8]	train-auc:0.822709	val-auc:0.771835
[9]	train-auc:0.824086	val-auc:0.772689
[10]	train-auc:0.824397	val-auc:0.774935
[11]	train-auc:0.826278	val-auc:0.776218
[12]	train-auc:0.827382	val-auc:0.776136
[13]	train-auc:0.827757	val-auc:0.777088
[14]	train-auc:0.82857	val-auc:0.777535
[15]	train-auc:0.828605	val-auc:0.778
[16]	train-auc:0.828837	val-auc:0.77854
[17]	train-auc:0.828966	val-auc:0.778484
[18]	train-auc:0.829693	val-auc:0.779021
[19]	train-auc:0.830318	val-auc:0.779322
[20]	train-auc:0.831009	val-auc:0.779649
[

KeyError: "['老段子特征x'] not in index"

In [17]:
print ("跑到这里了model.predict")
preds_offline = model.predict(xgb_offline_test,ntree_limit=model.best_iteration)#
preds_online = model.predict(xgb_online_test,ntree_limit=model.best_iteration)#
offline=offline_test[['用户标识','标签']]
online=online_test[['用户标识']]

offline['预测']=preds_offline
online['预测']=preds_online

print("线下得分;")
print(ks(offline.预测,offline.标签))

print ("跑到这里了,输出结果")
from sklearn.preprocessing import MinMaxScaler
online.预测 = MinMaxScaler().fit_transform(online.预测)
output=online[['用户标识','预测']].rename(index=str, columns={"用户标识": "userid", "预测": "probability"})
output.to_csv("../output/20170201_B.csv",index=None,encoding='utf-8')

#save feature score and feature information:  feature,score,min,max,n_null,n_gt1w
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
with open('../feature_score/feature_score_20170201_B.csv','w') as f:
        f.writelines("feature,score\n")
        f.writelines(fs)
        
cost_time = time.time()-start_time
print ("",'\n',"cost time:",cost_time,"(s)")
print ("跑到这里了,结束！")
# CV训练
#xgb.cv(xgb_config, dtrain, num_boost_round=10, nfold=5, maximize=False, verbose_eval=True)

跑到这里了model.predict


E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample

线下得分;
('ks', 0.49200187796347356)
跑到这里了,输出结果
 
 cost time: 1111.2796428203583 (s)
跑到这里了,结束！
